In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML


In [2]:
# Read the Excel file
file_path = "FINAL IFSC.xlsx"
df = pd.read_excel(file_path, dtype=str)  # Read all columns as text


In [3]:
# Display original data info
print("Original Data Info:")
print(f"Total records: {len(df)}")
print("\nFirst 5 records:")
display(df.head())

Original Data Info:
Total records: 65519

First 5 records:


,IFSCCODE,BANK,BRANCH,MICRNO,BRANCCODE,SWIFTCODE,ADDRESS,PINCODE,DISTRICT,STATE,CONTACT,EMAILID,CUSTOMERCARE
0,BARB0AAMBUR,BANK OF BARODA (BOB),AMBUR,NaN,AAMBUR,SWIFT Code Bank of Baroda (BOB),44 15A BYPASS ROAD AMBUR CHENNAI TAMIL NADU 63...,635802,CHENNAI,TAMIL NADU,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
1,BARB0AANTIY,BANK OF BARODA (BOB),AANTIYAA TALAB-UP,284012005,AANTIY,SWIFT Code Bank of Baroda (BOB),"1385, SHARDA HILLS COLONY, NEW BASTI, AANTIYAA...",284001,JHANSI,UTTAR PRADESH,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
2,BARB0AARANI,BANK OF BARODA (BOB),ARANI,NaN,AARANI,SWIFT Code Bank of Baroda (BOB),33 GANDHI ROAD OPP OLD BUS STAND ARANI DIST TH...,632301,TIRUVANNAMALAI,TAMIL NADU,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
3,BARB0ABADAN,BANK OF BARODA (BOB),ABADAN,NaN,ABADAN,SWIFT Code Bank of Baroda (BOB),"AT ABADAN, PO CHITRES, BLOCK KAKATPUR, DIST PU...",752108,PURI,ODISHA,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
4,BARB0ABHANP,BANK OF BARODA (BOB),ABHANPUR-RAIPUR-CH,492012503,ABHANP,SWIFT Code Bank of Baroda (BOB),"RAJIM ROAD, NEAR BUS STAND, ABHANPUR, DIST RAI...",497001,RAIPUR,CHHATTISGARH,9752410784,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."


In [4]:

## 1. Check for complete duplicate rows (all columns identical)
complete_duplicates = df[df.duplicated(keep=False)]
print(f"\n1. Complete Duplicates (all columns identical) - {len(complete_duplicates)} records")

if not complete_duplicates.empty:
    # Group duplicates together for better visualization
    dup_groups = complete_duplicates.groupby(list(df.columns)).size().reset_index(name='DUPLICATE_COUNT')
    display("Duplicate groups with counts:")
    display(dup_groups.sort_values('DUPLICATE_COUNT', ascending=False))
else:
    print("No complete duplicates found (all columns identical)")



1. Complete Duplicates (all columns identical) - 0 records
No complete duplicates found (all columns identical)


In [5]:
# 1. Find and eliminate duplicates using all columns
initial_count = len(df)
df.drop_duplicates(inplace=True)
after_dedup_count = len(df)
duplicates_removed = initial_count - after_dedup_count

print("\n1. Duplicate Removal:")
print(f"Removed {duplicates_removed} duplicate records")
print(f"Remaining records: {after_dedup_count}")


1. Duplicate Removal:
Removed 0 duplicate records
Remaining records: 65519


In [11]:
pincode_df = pd.read_excel("IFSC_PINCODE.xlsx", dtype=str)  # Assuming these column names


In [12]:
print("Original Data Info:")
print(f"Total records: {len(pincode_df)}")
print("\nFirst 5 records:")
display(pincode_df.head())

Original Data Info:
Total records: 13229

First 5 records:


,IFSCODE,BANK,BRANCH,PHONE,CITY,DISTRICT,STATE,PINCODE,ADDRESS
0,BARB0ADAHIS,Bank Of Baroda,Adahis,0172-5076839,Mandi Adampur,Hisar,Haryana,125052,"High school road, near rajendra hospital mandi..."
1,BARB0ADAHYD,Bank Of Baroda,Adarsh Nagar Hyderabad,040-23226982,Hyderabad,Hyderabad Urban,Andhra Pradesh,500063,"5-9-27/78 lakehill road, adarshnagar"
2,BARB0CHAKPR,Bank Of Baroda,Chakradharpur,08084378196-B N Kujur,Chakradharpur,Paschimi Singhbhum,Jharkhand,833102,"Ranchi, chaibasa,main road, chakradharpur, wes..."
3,BARB0ADITYA,Bank Of Baroda,Adityapur Jharkhand,9430183699,Adityapur,Saraikela Kharsawan,Jharkhand,831013,"Gr floor, m p naginacomp, tata kandra rd"
4,BARB0AGCPAT,Bank Of Baroda,A G Colony Patna Bihar,1800223344,Patna,Patna,Bihar,800023,"Main road,a g colony,patna,bihar – 800023."


In [13]:
# 3. Find duplicates based on IFSCCODE but differences in other fields
duplicate_ifsc = pincode_df[pincode_df.duplicated('IFSCODE', keep=False)].sort_values('IFSCODE')
conflicting_records = []

for ifsc_code in duplicate_ifsc['IFSCODE'].unique():
    group = duplicate_ifsc[duplicate_ifsc['IFSCODE'] == ifsc_code]
    if len(group) > 1 and not group.iloc[1:].equals(group.iloc[:-1]):
        conflicting_records.append(group)

print("\n3. Conflicting Records (same IFSC but different other fields):")
if conflicting_records:
    print(f"Found {len(conflicting_records)} IFSC codes with conflicting data")
    
    # Highlight function for conflicting records
    def highlight_conflicts(s):
        is_dupe = s.index.duplicated(keep=False)
        return ['background-color: red' if v else '' for v in is_dupe]
    
    # Display first 5 conflicts if they exist
    conflicts_df = pd.concat(conflicting_records[:])
    display("Sample conflicting records (highlighted in red):")
    display(conflicts_df) #.style.apply(highlight_conflicts, axis=0))
else:
    print("No conflicting records found (same IFSC with different other fields)")


3. Conflicting Records (same IFSC but different other fields):
No conflicting records found (same IFSC with different other fields)


In [14]:
main_df = pd.read_excel("FINAL IFSC.xlsx", dtype=str)

In [15]:
# Display info about both datasets
print("Main Dataset Info:")
print(f"Records: {len(main_df)}")
print(f"Missing PINCODEs: {main_df['PINCODE'].isna().sum() + (main_df['PINCODE'] == '').sum()}")


Main Dataset Info:
Records: 65519
Missing PINCODEs: 13564


In [17]:
print("\nPincode Reference Dataset Info:")
print(f"Records: {len(pincode_df)}")
print(f"Unique IFSC codes: {pincode_df['IFSCODE'].nunique()}")



Pincode Reference Dataset Info:
Records: 13229
Unique IFSC codes: 13224


In [18]:
# Clean PINCODE data - remove empty/whitespace
pincode_df['PINCODE'] = pincode_df['PINCODE'].str.strip()
pincode_df = pincode_df.dropna(subset=['PINCODE'])
pincode_df = pincode_df[pincode_df['PINCODE'] != '']


In [20]:
# Create a clean mapping dictionary (IFSC to PINCODE)
ifsc_pincode_map = pincode_df.drop_duplicates('IFSCODE', keep='last').set_index('IFSCODE')['PINCODE'].to_dict()


In [21]:


# Function to fill missing pincodes
def fill_pincode(row):
    if pd.isna(row['PINCODE']) or row['PINCODE'] == '':
        return ifsc_pincode_map.get(row['IFSCCODE'], row['PINCODE'])
    return row['PINCODE']



In [22]:
# Create a copy of the original dataframe
merged_df = main_df.copy()

# Fill missing pincodes
merged_df['PINCODE'] = merged_df.apply(fill_pincode, axis=1)


In [24]:
# Calculate statistics
initial_missing = main_df['PINCODE'].isna().sum() + (main_df['PINCODE'] == '').sum()
final_missing = merged_df['PINCODE'].isna().sum() + (merged_df['PINCODE'] == '').sum()
filled_count = initial_missing - final_missing
# Display results
print("\nMerge Results:")
print(f"Originally missing PINCODEs: {initial_missing}")
print(f"Filled PINCODEs from reference: {filled_count}")
print(f"Remaining missing PINCODEs: {final_missing}")



Merge Results:
Originally missing PINCODEs: 13564
Filled PINCODEs from reference: 11095
Remaining missing PINCODEs: 2469


In [25]:
# 2. Create clean dataframe by removing these records
merged_count = len(merged_df)
clean_df = merged_df[~(merged_df['PINCODE'].isna() | (merged_df['PINCODE'] == ''))].copy()
clean_count = len(clean_df)
print(f"before count: {merged_count}")
print(f"after count: {clean_count}")

before count: 65519
after count: 63050


In [26]:

# 3. Save the clean data to a new Excel file
clean_file_path = "FINAL_IFSC_CLEAN.csv"
clean_df.to_csv(clean_file_path, index=False)


In [27]:
clean_df.head(10)

,IFSCCODE,BANK,BRANCH,MICRNO,BRANCCODE,SWIFTCODE,ADDRESS,PINCODE,DISTRICT,STATE,CONTACT,EMAILID,CUSTOMERCARE
0,BARB0AAMBUR,BANK OF BARODA (BOB),AMBUR,NaN,AAMBUR,SWIFT Code Bank of Baroda (BOB),44 15A BYPASS ROAD AMBUR CHENNAI TAMIL NADU 63...,635802,CHENNAI,TAMIL NADU,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
1,BARB0AANTIY,BANK OF BARODA (BOB),AANTIYAA TALAB-UP,284012005,AANTIY,SWIFT Code Bank of Baroda (BOB),"1385, SHARDA HILLS COLONY, NEW BASTI, AANTIYAA...",284001,JHANSI,UTTAR PRADESH,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
2,BARB0AARANI,BANK OF BARODA (BOB),ARANI,NaN,AARANI,SWIFT Code Bank of Baroda (BOB),33 GANDHI ROAD OPP OLD BUS STAND ARANI DIST TH...,632301,TIRUVANNAMALAI,TAMIL NADU,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
3,BARB0ABADAN,BANK OF BARODA (BOB),ABADAN,NaN,ABADAN,SWIFT Code Bank of Baroda (BOB),"AT ABADAN, PO CHITRES, BLOCK KAKATPUR, DIST PU...",752108,PURI,ODISHA,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
4,BARB0ABHANP,BANK OF BARODA (BOB),ABHANPUR-RAIPUR-CH,492012503,ABHANP,SWIFT Code Bank of Baroda (BOB),"RAJIM ROAD, NEAR BUS STAND, ABHANPUR, DIST RAI...",497001,RAIPUR,CHHATTISGARH,9752410784,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
5,BARB0ABHAYK,BANK OF BARODA (BOB),ABHAYKHAND-GHAZIABAD-UP,NaN,ABHAYK,SWIFT Code Bank of Baroda (BOB),"MAHALAXMI SQUARE, C1 ABHAY KHAND-2, INDIRAPURA...",201010,GHAZIABAD,UTTAR PRADESH,0120-4348055,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
6,BARB0ABOHAR,BANK OF BARODA (BOB),ABOHAR-DIST FEROZPUR,152012302,ABOHAR,SWIFT Code Bank of Baroda (BOB),"ST.NO.9, ABOHAR DIST FEROZPUR 152116",152116,FIROZPUR,PUNJAB,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
7,BARB0ABRAMA,BANK OF BARODA (BOB),ABRAMA-DIST NAVSARI,396012509,ABRAMA,SWIFT Code Bank of Baroda (BOB),"AT & PO.:ABRAMA, TA.:JALALPORE, DIST NAVSARI, ...",396406,NAVSARI,GUJARAT,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
8,BARB0ABRBUL,BANK OF BARODA (BOB),ABRAMA BR-VALSAD-GUJARAT,396012007,ABRBUL,SWIFT Code Bank of Baroda (BOB),"H NO 16/2981 DHARAMPUR ROAD, VALSAD, GUJARAT-3...",396001,VALSAD,GUJARAT,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
9,BARB0ABROAD,BANK OF BARODA (BOB),ABU ROAD-DIST SIROHI,307012025,ABROAD,SWIFT Code Bank of Baroda (BOB),"ABU ROAD, DURGA HOTEL COMPOUNDPARSI CHAWL DIST...",307026,SIROHI,RAJASTHAN,NaN,abroad@bankofbaroda.com,"Toll free number 1800 22 33 44, 1800 102 44 55..."


In [28]:
# 2. Find rows with missing PINCODE and highlight them
missing_pincode = df[df['PINCODE'].isna() | (df['PINCODE'] == '')]
print("\n2. Rows with Missing PINCODE:")
print(f"Count: {len(missing_pincode)}")

def highlight_missing_pincode(row):
    if pd.isna(row['PINCODE']) or row['PINCODE'] == '':
        return ['background-color: red'] * len(row)
    return [''] * len(row)

if not missing_pincode.empty:
    display("Sample rows with missing PINCODE (highlighted in red):")
    display(df[df['PINCODE'].isna() | (df['PINCODE'] == '')]) #.style.apply(highlight_missing_pincode, axis=1))
else:
    print("No records with missing PINCODE found.")
    
# Optional: Save the conflicting records to a new Excel file
#if conflict_groups:
(df[df['PINCODE'].isna() | (df['PINCODE'] == '')]).to_excel("missing_pincode.xlsx", index=False)
#    print("\nConflicting records saved to 'branccode_bank_conflicts.xlsx'")


2. Rows with Missing PINCODE:
Count: 13564


'Sample rows with missing PINCODE (highlighted in red):'

,IFSCCODE,BANK,BRANCH,MICRNO,BRANCCODE,SWIFTCODE,ADDRESS,PINCODE,DISTRICT,STATE,CONTACT,EMAILID,CUSTOMERCARE
14,BARB0ADAHIS,BANK OF BARODA (BOB),ADAHIS,NaN,ADAHIS,SWIFT Code Bank of Baroda (BOB),"HIGH SCHOOL ROAD, NEAR RAJENDRA HOSPITAL MANDI...",NaN,HISAR,HARYANA,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
15,BARB0ADAHYD,BANK OF BARODA (BOB),ADARSH NAGAR-HYDERABAD,500012016,ADAHYD,SWIFT Code Bank of Baroda (BOB),"5-9-27/78 LAKEHILL ROAD, ADARSHNAGAR TELANGANA",NaN,HYDERABAD,TELANGANA,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
28,BARB0ADITYA,BANK OF BARODA (BOB),ADITYAPUR-JHARKHAND,NaN,ADITYA,SWIFT Code Bank of Baroda (BOB),"GR FLOOR, M P NAGINACOMP, TATA KANDRA RD",NaN,SERAIKELLA KHARSAWAN,JHARKHAND,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
37,BARB0AGCPAT,BANK OF BARODA (BOB),A G COLONY-PATNA,800012018,AGCPAT,SWIFT Code Bank of Baroda (BOB),"MAIN ROAD, A G COLONY, PATNA, BIHAR",NaN,PATNA,BIHAR,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
49,BARB0AHRABS,BANK OF BARODA (BOB),AHRAURA-UP,231012051,AHRABS,SWIFT Code Bank of Baroda (BOB),"CHOWK BAZAR, AHRAURA, RO VARANASI, UP",NaN,MIRZAPUR,UTTAR PRADESH,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65505,UCBA0003347,UCO BANK,VALANDIPALAYAM,NaN,003347,SWIFT Code UCO Bank,203 THADAGAM BRANCH VALANDIPALAYAM COIMBATORE,NaN,COIMBATORE,TAMIL NADU,NaN,NaN,1800 103 0123\n uco.custcare@ucobank.co.in
65506,UCBA0003348,UCO BANK,EAST PRATAPGARH,NaN,003348,SWIFT Code UCO Bank,VILL BANKUMARI BAZAR PO JOGENDRANAGARH,NaN,WEST TRIPURA,TRIPURA,NaN,NaN,1800 103 0123\n uco.custcare@ucobank.co.in
65510,UCBA0003352,UCO BANK,BADGAON,494028503,003352,SWIFT Code UCO Bank,"771, GRAMIN SACHIVALAYA, VILLAGE BADGAON, P.O....",NaN,KANKER,CHHATTISGARH,NaN,NaN,1800 103 0123\n uco.custcare@ucobank.co.in
65516,UCBA0RRBBKG,UCO BANK,BIHAR KSHETRIYA GRAMIN BANK-RRB,NaN,RRBBKG,SWIFT Code UCO Bank,"HEAD OFFICE, BHAGAT SINGH CHOWK MUNGER, BIHAR",NaN,MUNGER,BIHAR,06344-222055,NaN,1800 103 0123\n uco.custcare@ucobank.co.in


In [40]:
# 1. Identify records with missing PINCODE
missing_pincode = df[df['PINCODE'].isna() | (df['PINCODE'] == '')]
missing_count = len(missing_pincode)

# 2. Create clean dataframe by removing these records
clean_df = df[~(df['PINCODE'].isna() | (df['PINCODE'] == ''))].copy()
clean_count = len(clean_df)

# 3. Save the clean data to a new Excel file
clean_file_path = "FINAL_IFSC_CLEAN.csv"
clean_df.head(5000).to_csv(clean_file_path, index=False)

# 4. Display summary of cleaning process
print("Data Cleaning Summary:")
print(f"Initial number of records: {initial_count}")
print(f"Records with missing PINCODE: {missing_count}")
print(f"Remaining clean records: {clean_count}")
print(f"\nClean data saved to: {clean_file_path}")

# 5. Show sample of removed records (if any)
if not missing_pincode.empty:
    print("\nSample of removed records (missing PINCODE):")
    display(missing_pincode.head())
else:
    print("\nNo records with missing PINCODE found.")

# 6. Show sample of clean data
print("\nSample of clean data:")
display(clean_df.head())

Data Cleaning Summary:
Initial number of records: 65519
Records with missing PINCODE: 13564
Remaining clean records: 51955

Clean data saved to: FINAL_IFSC_CLEAN.csv

Sample of removed records (missing PINCODE):


,IFSCCODE,BANK,BRANCH,MICRNO,BRANCCODE,SWIFTCODE,ADDRESS,PINCODE,DISTRICT,STATE,CONTACT,EMAILID,CUSTOMERCARE
14,BARB0ADAHIS,BANK OF BARODA (BOB),ADAHIS,NaN,ADAHIS,SWIFT Code Bank of Baroda (BOB),"HIGH SCHOOL ROAD, NEAR RAJENDRA HOSPITAL MANDI...",NaN,HISAR,HARYANA,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
15,BARB0ADAHYD,BANK OF BARODA (BOB),ADARSH NAGAR-HYDERABAD,500012016,ADAHYD,SWIFT Code Bank of Baroda (BOB),"5-9-27/78 LAKEHILL ROAD, ADARSHNAGAR TELANGANA",NaN,HYDERABAD,TELANGANA,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
28,BARB0ADITYA,BANK OF BARODA (BOB),ADITYAPUR-JHARKHAND,NaN,ADITYA,SWIFT Code Bank of Baroda (BOB),"GR FLOOR, M P NAGINACOMP, TATA KANDRA RD",NaN,SERAIKELLA KHARSAWAN,JHARKHAND,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
37,BARB0AGCPAT,BANK OF BARODA (BOB),A G COLONY-PATNA,800012018,AGCPAT,SWIFT Code Bank of Baroda (BOB),"MAIN ROAD, A G COLONY, PATNA, BIHAR",NaN,PATNA,BIHAR,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
49,BARB0AHRABS,BANK OF BARODA (BOB),AHRAURA-UP,231012051,AHRABS,SWIFT Code Bank of Baroda (BOB),"CHOWK BAZAR, AHRAURA, RO VARANASI, UP",NaN,MIRZAPUR,UTTAR PRADESH,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."



Sample of clean data:


,IFSCCODE,BANK,BRANCH,MICRNO,BRANCCODE,SWIFTCODE,ADDRESS,PINCODE,DISTRICT,STATE,CONTACT,EMAILID,CUSTOMERCARE
0,BARB0AAMBUR,BANK OF BARODA (BOB),AMBUR,NaN,AAMBUR,SWIFT Code Bank of Baroda (BOB),44 15A BYPASS ROAD AMBUR CHENNAI TAMIL NADU 63...,635802,CHENNAI,TAMIL NADU,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
1,BARB0AANTIY,BANK OF BARODA (BOB),AANTIYAA TALAB-UP,284012005,AANTIY,SWIFT Code Bank of Baroda (BOB),"1385, SHARDA HILLS COLONY, NEW BASTI, AANTIYAA...",284001,JHANSI,UTTAR PRADESH,NaN,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
2,BARB0AARANI,BANK OF BARODA (BOB),ARANI,NaN,AARANI,SWIFT Code Bank of Baroda (BOB),33 GANDHI ROAD OPP OLD BUS STAND ARANI DIST TH...,632301,TIRUVANNAMALAI,TAMIL NADU,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
3,BARB0ABADAN,BANK OF BARODA (BOB),ABADAN,NaN,ABADAN,SWIFT Code Bank of Baroda (BOB),"AT ABADAN, PO CHITRES, BLOCK KAKATPUR, DIST PU...",752108,PURI,ODISHA,1800223344,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."
4,BARB0ABHANP,BANK OF BARODA (BOB),ABHANPUR-RAIPUR-CH,492012503,ABHANP,SWIFT Code Bank of Baroda (BOB),"RAJIM ROAD, NEAR BUS STAND, ABHANPUR, DIST RAI...",497001,RAIPUR,CHHATTISGARH,9752410784,NaN,"Toll free number 1800 22 33 44, 1800 102 44 55..."


In [22]:
# 3. Find duplicates based on IFSCCODE but differences in other fields
duplicate_ifsc = df[df.duplicated('IFSCCODE', keep=False)].sort_values('IFSCCODE')
conflicting_records = []

for ifsc_code in duplicate_ifsc['IFSCCODE'].unique():
    group = duplicate_ifsc[duplicate_ifsc['IFSCCODE'] == ifsc_code]
    if len(group) > 1 and not group.iloc[1:].equals(group.iloc[:-1]):
        conflicting_records.append(group)

print("\n3. Conflicting Records (same IFSC but different other fields):")
if conflicting_records:
    print(f"Found {len(conflicting_records)} IFSC codes with conflicting data")
    
    # Highlight function for conflicting records
    def highlight_conflicts(s):
        is_dupe = s.index.duplicated(keep=False)
        return ['background-color: red' if v else '' for v in is_dupe]
    
    # Display first 5 conflicts if they exist
    conflicts_df = pd.concat(conflicting_records[:])
    display("Sample conflicting records (highlighted in red):")
    display(conflicts_df) #.style.apply(highlight_conflicts, axis=0))
else:
    print("No conflicting records found (same IFSC with different other fields)")


3. Conflicting Records (same IFSC but different other fields):
No conflicting records found (same IFSC with different other fields)


In [24]:
def highlight_conflicts(s):
    return ['background-color: red' if s.name in conflict_indices else '' for _ in s]

# 1. Find duplicates based on BRANCCODE and BANK
duplicates = df[df.duplicated(['BRANCCODE', 'BANK'], keep=False)].sort_values(['BRANCCODE', 'BANK'])

# 2. Identify records with same BRANCCODE+BANK but different other fields
conflict_groups = []
conflict_indices = []

for (branccode, bank), group in duplicates.groupby(['BRANCCODE', 'BANK']):
    # Check if there are differences in other columns
    if not group.drop(['BRANCCODE', 'BANK'], axis=1).apply(pd.Series.duplicated, keep=False).all().all():
        conflict_groups.append(group)
        conflict_indices.extend(group.index.tolist())

# 3. Display results
print(f"Found {len(conflict_groups)} groups with same BRANCCODE+BANK but different other fields")

if conflict_groups:
    # Combine all conflicting records
    all_conflicts = pd.concat(conflict_groups)
    
    # Display summary count
    conflict_summary = all_conflicts.groupby(['BRANCCODE', 'BANK']).size().reset_index(name='COUNT')
    print("\nSummary of conflicting groups:")
    display(conflict_summary.sort_values('COUNT', ascending=False))
    
    # Display sample conflicts (first 10 records) highlighted
    print("\nSample conflicting records (highlighted in red):")
    display(df.loc[all_conflicts.index[:10]]) #.style.apply(highlight_conflicts, axis=1))
else:
    print("No conflicting records found with same BRANCCODE+BANK but different other fields")

# Optional: Save the conflicting records to a new Excel file
#if conflict_groups:
#    pd.concat(conflict_groups).to_excel("branccode_bank_conflicts.xlsx", index=False)
#    print("\nConflicting records saved to 'branccode_bank_conflicts.xlsx'")

Found 0 groups with same BRANCCODE+BANK but different other fields
No conflicting records found with same BRANCCODE+BANK but different other fields


In [25]:

# 4. Summary by BANK
bank_summary = df['BANK'].value_counts().reset_index()
bank_summary.columns = ['BANK', 'COUNT']
print("\n4. Record Count by BANK:")
display(bank_summary)


4. Record Count by BANK:


,BANK,COUNT
0,STATE BANK OF INDIA (SBI),26088
1,CANARA BANK,6516
2,BANK OF BARODA (BOB),5497
3,BANK OF INDIA (BOI),5295
4,CENTRAL BANK OF INDIA,4938
5,UNION BANK OF INDIA,4486
6,INDIAN OVERSEAS BANK (IOB),3463
7,UCO BANK,3100
8,INDIAN BANK,2694
9,BANK OF MAHARASHTRA,1924


In [26]:
# 5. Summary by STATE
state_summary = df['STATE'].value_counts().reset_index()
state_summary.columns = ['STATE', 'COUNT']
print("\n5. Record Count by STATE:")
display(state_summary)



5. Record Count by STATE:


,STATE,COUNT
0,UTTAR PRADESH,7393
1,MAHARASHTRA,6940
2,TAMIL NADU,5661
3,KARNATAKA,4167
4,GUJARAT,4104
5,WEST BENGAL,3552
6,MADHYA PRADESH,3424
7,RAJASTHAN,3146
8,ANDHRA PRADESH,2992
9,KERALA,2951


In [27]:

# 6. Summary by BANK, STATE, DISTRICT
bank_state_district_summary = df.groupby(['BANK', 'STATE', 'DISTRICT']).size().reset_index(name='COUNT')
print("\n6. Record Count by BANK, STATE, DISTRICT:")
display(bank_state_district_summary.sort_values(['BANK', 'STATE', 'COUNT'], ascending=[True, True, False]))


6. Record Count by BANK, STATE, DISTRICT:


,BANK,STATE,DISTRICT,COUNT
0,BANK OF BARODA (BOB),ANDAMAN & NICOBAR,ANDAMAN,1
3,BANK OF BARODA (BOB),ANDHRA PRADESH,EAST GODAVARI,24
10,BANK OF BARODA (BOB),ANDHRA PRADESH,VISAKHAPATNAM,18
4,BANK OF BARODA (BOB),ANDHRA PRADESH,GUNTUR,14
5,BANK OF BARODA (BOB),ANDHRA PRADESH,KRISHNA,13
...,...,...,...,...
5433,UNION BANK OF INDIA,WEST BENGAL,BANKURA,3
5436,UNION BANK OF INDIA,WEST BENGAL,COOCH BEHAR,3
5451,UNION BANK OF INDIA,WEST BENGAL,UTTAR DINAJPUR,3
5447,UNION BANK OF INDIA,WEST BENGAL,PASCHIM MEDINIPUR,2
